script to use the dalitz plot method outlined by matt shepard to calculate the efficiency of the kstar rejection in the KKPi phasespace

The idea would be to correct the data by this as opposed to the phasespace acceptance, as this just handles the K* rejection and not the overall efficiency (which may not actually model the f1 well, unlike the signal MC)

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import ROOT
import numpy as np
import pandas as pd
from my_library.common_analysis_tools import get_integrated_data_hist, COLORBLIND_HEX_DICT, get_integrated_gluex1_acceptance_corrected_data, get_integrated_gluex1_data

Welcome to JupyROOT 6.24/04


In [2]:
mass_low = 1.0 # GeV
mass_high = 2.5
step_size = 0.01


kstar_mass_upper = 1.0
kstar_mass_lower = 0.8

kaon_mass = 0.493677
kshort_mass = 0.497611
pion_mass = 0.139570


In [3]:
def check_if_kstar_region(mass):
    return mass < kstar_mass_upper and mass > kstar_mass_lower


def get_weight_per_mass(mass_bin, draw_plots=False):
    phasespace_centermass = ROOT.TLorentzVector(0, 0, 0, mass_bin)
    phasespace_generator = ROOT.TGenPhaseSpace()
    phasespace_generator.SetDecay(phasespace_centermass, 3, np.array([kaon_mass, kshort_mass, pion_mass]))

    total = 0
    passed = 0

    full_dalitz = ROOT.TH2D(f"dalitz_{mass_bin}", f"dalitz_{mass_bin}", 100, 0, 2, 100, 0, 2)
    passed_dalitz = ROOT.TH2D(f"passed_dalitz_{mass_bin}", f"passed_dalitz_{mass_bin}", 100, 0, 2, 100, 0, 2)

    for _ in range(100000):
        weight = phasespace_generator.Generate()
        kaon = phasespace_generator.GetDecay(0)
        kshort = phasespace_generator.GetDecay(1)
        pion = phasespace_generator.GetDecay(2)

        neutral_kstar_mass = (kaon + pion).M()
        charged_kstar_mass = (kshort + pion).M()

        full_dalitz.Fill(neutral_kstar_mass*neutral_kstar_mass, charged_kstar_mass*charged_kstar_mass, weight)
        total += weight

        if check_if_kstar_region(neutral_kstar_mass) or check_if_kstar_region(charged_kstar_mass):
            continue 

        passed_dalitz.Fill(neutral_kstar_mass*neutral_kstar_mass, charged_kstar_mass*charged_kstar_mass, weight)
        passed += weight

    if draw_plots:
        c = ROOT.TCanvas()
        c.Divide(2,1)
        c.cd(1)
        full_dalitz.Draw("colz")
        c.cd(2)
        passed_dalitz.Draw("colz")
        c.Update()
        c.SaveAs("/work/halld/home/viducic/plots/ps_dalitz/dalitz_{}.png".format(mass_bin))
        c.Close()


    return passed / total

In [4]:
mass_bin_centers = []
kstar_cut_efficiencies = []
for mass_bin in np.arange(mass_low, mass_high+step_size, step_size):
    mass_bin_center = round(mass_bin + (step_size/2), 4)
    print(mass_bin_center)
    binned_weight = get_weight_per_mass(mass_bin_center)
    mass_bin_centers.append(mass_bin_center)
    kstar_cut_efficiencies.append(binned_weight)


1.005
1.015
1.025
1.035
1.045
1.055
1.065
1.075
1.085
1.095
1.105
1.115
1.125
1.135
1.145
1.155
1.165
1.175
1.185
1.195
1.205
1.215
1.225
1.235
1.245
1.255
1.265
1.275
1.285
1.295
1.305
1.315
1.325
1.335
1.345
1.355
1.365
1.375
1.385
1.395
1.405
1.415
1.425
1.435
1.445
1.455
1.465
1.475
1.485
1.495
1.505
1.515
1.525
1.535
1.545
1.555
1.565
1.575
1.585
1.595
1.605
1.615
1.625
1.635
1.645
1.655
1.665
1.675
1.685
1.695
1.705
1.715
1.725
1.735
1.745
1.755
1.765
1.775
1.785
1.795
1.805
1.815
1.825
1.835
1.845
1.855
1.865
1.875
1.885
1.895
1.905
1.915
1.925
1.935
1.945
1.955
1.965
1.975
1.985
1.995
2.005
2.015
2.025
2.035
2.045
2.055
2.065
2.075
2.085
2.095
2.105
2.115
2.125
2.135
2.145
2.155
2.165
2.175
2.185
2.195
2.205
2.215
2.225
2.235
2.245
2.255
2.265
2.275
2.285
2.295
2.305
2.315
2.325
2.335
2.345
2.355
2.365
2.375
2.385
2.395
2.405
2.415
2.425
2.435
2.445
2.455
2.465
2.475
2.485
2.495
2.505


In [5]:
df = pd.DataFrame({"mass_bin_center": mass_bin_centers, "kstar_cut_efficiency": kstar_cut_efficiencies})
df['kstar_cut_efficiency'] = df['kstar_cut_efficiency'].fillna(1)
df.to_csv(f"/work/halld/home/viducic/data/ps_dalitz/kstar_cut_efficiency_{step_size*1000}.csv", index=False)

In [6]:
def apply_efficiency_correction(hist):
    corrected_hist = hist.Clone()
    for i in range(1, corrected_hist.GetNbinsX()+1):
        mass_bin_center = round(hist.GetBinCenter(i),3)
        eff_df = df[df["mass_bin_center"] == round(mass_bin_center, 3)]["kstar_cut_efficiency"]
        if len(eff_df) == 0:
            continue
        efficiency = eff_df.values[0]
        corrected_hist.SetBinContent(i, hist.GetBinContent(i) / efficiency)
    
    corrected_hist.SetDirectory(0)

    return corrected_hist

In [7]:
hist = get_integrated_gluex1_data('pipkmks', 'all')
hist.SetLineColor(ROOT.TColor.GetColor(COLORBLIND_HEX_DICT['blue']))
corrected_hist = apply_efficiency_correction(hist)
corrected_hist.SetLineColor(ROOT.TColor.GetColor(COLORBLIND_HEX_DICT['red']))
uncut_hist = get_integrated_gluex1_data('pipkmks', "no")
uncut_hist.SetLineColor(ROOT.TColor.GetColor(COLORBLIND_HEX_DICT['purple']))
uncut_hist.SetTitle("M(#pi^{+}K^{-}K_{s}) with and without K* rejection efficiency correction")
hist.SetTitle("M(#pi^{+}K^{-}K_{s}) with and without K* rejection efficiency correction")
acceptance_corrected_hist = get_integrated_gluex1_acceptance_corrected_data('pipkmks', 'all')
acceptance_corrected_hist.SetLineColor(ROOT.TColor.GetColor(COLORBLIND_HEX_DICT['green']))

acceptance_corrected_hist.Scale(1/44)

fit_range_min = 1.2
fit_range_max = 1.5
hist.GetXaxis().SetRangeUser(fit_range_min, fit_range_max)
uncut_hist.GetXaxis().SetRangeUser(fit_range_min, fit_range_max)
corrected_hist.GetXaxis().SetRangeUser(fit_range_min, fit_range_max)
acceptance_corrected_hist.GetXaxis().SetRangeUser(fit_range_min, fit_range_max)



AttributeError: 'TObject' object has no attribute 'SetDirectory'

In [ ]:
legend = ROOT.TLegend(0.6, 0.6, 0.9, 0.5)
legend.AddEntry(hist, "K* rejection without correction", "l")
legend.AddEntry(corrected_hist, "K* rejection with correction", "l")
# legend.AddEntry(uncut_hist, "No K* rejection", "l")
legend.AddEntry(acceptance_corrected_hist, "Acceptance corrected", "l")

In [ ]:
c1 = ROOT.TCanvas()
# uncut_hist.Draw()
hist.Draw()
corrected_hist.Draw("same")
acceptance_corrected_hist.Draw("same")
legend.Draw()
c1.Update()
c1.Draw()